In [ ]:
#hide
from brown_phototour_revisited.dataset import *
#from brown_phototour_revisited.extraction import *
from brown_phototour_revisited.evaluation import *

#  brown_phototour_revisited

> The package for local patch descriptors evaluation, which takes into account image indexes and second nearest neighbor ratio (SNN) filtering strategy. It is in agreement with IMC benchmark and practice, unlike the original protocol

This file will become your README and also the index of your documentation.

## Install

`pip install brown_phototour_revisited`

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2